# Tune GPT2 to generate positive reviews
> Optimise GPT2 to produce positive IMDB movie reviews using a BERT sentiment classifier as a reward function.

<div style="text-align: center">
<img src='https://huggingface.co/datasets/trl-internal-testing/example-images/resolve/main/images/gpt2_bert_training.png' width='600'>
<p style="text-align: center;"> <b>Figure:</b> Experiment setup to tune GPT2. The yellow arrows are outside the scope of this notebook, but the trained models are available through Hugging Face. </p>
</div>


In this notebook we fine-tune GPT2 (small) to generate positive movie reviews based on the IMDB dataset. The model gets the start of a real review and is tasked to produce positive continuations. To reward positive continuations we use a BERT classifier to analyse the sentiment of the produced sentences and use the classifier's outputs as rewards signals for PPO training.

## Setup experiment

### Import dependencies

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch

In [3]:
torch.cuda.is_available()

True

In [4]:
torch.cuda.current_device()

0

In [5]:
torch.cuda.get_device_name(0)

'NVIDIA GeForce RTX 3050 Laptop GPU'

In [6]:
# %pip install transformers trl wandb

In [7]:
import torch
from tqdm import tqdm
import pandas as pd
import trl 


tqdm.pandas()

from transformers import pipeline, AutoTokenizer
from datasets import load_dataset

from trl import PPOTrainer, PPOConfig, AutoModelForCausalLMWithValueHead
from trl.core import LengthSampler

c:\Users\arvin\anaconda3\envs\dh_lab\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Configuration

In [8]:
config = PPOConfig(
    model_name="lvwerra/gpt2-imdb",
    learning_rate=1.41e-5,
    log_with="wandb",
)

sent_kwargs = {"return_all_scores": True, "function_to_apply": "none", "batch_size": 16}

In [9]:
import wandb

wandb.init()

wandb: Currently logged in as: arvind-menon (dh_lab). Use `wandb login --relogin` to force relogin


You can see that we load a GPT2 model called `gpt2_imdb`. This model was additionally fine-tuned on the IMDB dataset for 1 epoch with the huggingface [script](https://github.com/huggingface/transformers/blob/master/examples/run_language_modeling.py) (no special settings). The other parameters are mostly taken from the original paper ["Fine-Tuning Language Models from Human Preferences"](
https://arxiv.org/pdf/1909.08593.pdf). This model as well as the BERT model is available in the Huggingface model zoo [here](https://huggingface.co/models). The following code should automatically download the models.

## Load data and models

### Load IMDB dataset
The IMDB dataset contains 50k movie review annotated with "positive"/"negative" feedback indicating the sentiment.  We load the IMDB dataset into a DataFrame and filter for comments that are at least 200 characters. Then we tokenize each text and cut it to random size with the `LengthSampler`.

In [10]:
def build_dataset(config, dataset_name="imdb", input_min_text_length=2, input_max_text_length=8):
    """
    Build dataset for training. This builds the dataset from `load_dataset`, one should
    customize this function to train the model on its own dataset.

    Args:
        dataset_name (`str`):
            The name of the dataset to be loaded.

    Returns:
        dataloader (`torch.utils.data.DataLoader`):
            The dataloader for the dataset.
    """
    tokenizer = AutoTokenizer.from_pretrained(config.model_name)
    tokenizer.pad_token = tokenizer.eos_token
    # load imdb with datasets
    ds = load_dataset(dataset_name, split="train")
    ds = ds.rename_columns({"text": "review"})
    ds = ds.filter(lambda x: len(x["review"]) > 200, batched=False)

    input_size = LengthSampler(input_min_text_length, input_max_text_length)

    def tokenize(sample):
        sample["input_ids"] = tokenizer.encode(sample["review"])[: input_size()]
        sample["query"] = tokenizer.decode(sample["input_ids"])
        return sample

    ds = ds.map(tokenize, batched=False)
    ds.set_format(type="torch")
    return ds

In [11]:
dataset = build_dataset(config)


def collator(data):
    return dict((key, [d[key] for d in data]) for key in data[0])

### Load pre-trained GPT2 language models

We load the GPT2 model with a value head and the tokenizer. We load the model twice; the first model is optimized while the second model serves as a reference to calculate the KL-divergence from the starting point. This serves as an additional reward signal in the PPO training to make sure the optimized model does not deviate too much from the original language model.

In [12]:
model = AutoModelForCausalLMWithValueHead.from_pretrained(config.model_name)
ref_model = AutoModelForCausalLMWithValueHead.from_pretrained(config.model_name)
tokenizer = AutoTokenizer.from_pretrained(config.model_name)

tokenizer.pad_token = tokenizer.eos_token

### Initialize PPOTrainer
The `PPOTrainer` takes care of device placement and optimization later on:

In [13]:
ppo_trainer = PPOTrainer(config, model, ref_model, tokenizer, dataset=dataset, data_collator=collator)

### Load BERT classifier
We load a BERT classifier fine-tuned on the IMDB dataset.

In [37]:

device = ppo_trainer.accelerator.device
if ppo_trainer.accelerator.num_processes == 1:
    device = 0 if torch.cuda.is_available() else "cpu"  # to avoid a `pipeline` bug
sentiment_pipe = pipeline("sentiment-analysis", model="lvwerra/distilbert-imdb", device=device)
sentiment_pipe1 = pipeline("sentiment-analysis", model="omidroshani/imdb-sentiment-analysis", device=device)

config.json: 100%|██████████| 953/953 [00:00<00:00, 318kB/s]
c:\Users\arvin\anaconda3\envs\dh_lab\lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\arvin\.cache\huggingface\hub\models--nlptown--bert-base-multilingual-uncased-sentiment. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
pytorch_model.bin:

The model outputs are the logits for the negative and positive class. We will use the logits for positive class as a reward signal for the language model.

In [40]:
text = "this movie was really bad!!"
print(sentiment_pipe(text, **sent_kwargs))
print(sentiment_pipe1(text, **sent_kwargs))


[[{'label': 'NEGATIVE', 'score': 2.3350484371185303}, {'label': 'POSITIVE', 'score': -2.726576328277588}]]
[[{'label': 'LABEL_0', 'score': 1.645775556564331}, {'label': 'LABEL_1', 'score': -1.543138027191162}]]


c:\Users\arvin\anaconda3\envs\dh_lab\lib\site-packages\transformers\pipelines\text_classification.py:105: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [41]:
text = "this movie was really good!!"
print(sentiment_pipe(text, **sent_kwargs))
print(sentiment_pipe1(text, **sent_kwargs))


[[{'label': 'NEGATIVE', 'score': -2.294790267944336}, {'label': 'POSITIVE', 'score': 2.557040214538574}]]
[[{'label': 'LABEL_0', 'score': -1.2686774730682373}, {'label': 'LABEL_1', 'score': 1.1461920738220215}]]


### Generation settings
For the response generation we just use sampling and make sure top-k and nucleus sampling are turned off as well as a minimal length.

In [17]:
gen_kwargs = {"min_length": -1, "top_k": 0.0, "top_p": 1.0, "do_sample": True, "pad_token_id": tokenizer.eos_token_id}

## Optimize model

### Training loop

The training loop consists of the following main steps:
1. Get the query responses from the policy network (GPT-2)
2. Get sentiments for query/responses from BERT
3. Optimize policy with PPO using the (query, response, reward) triplet

**Training time**

This step takes **~2h** on a V100 GPU with the above specified settings.

In [63]:
output_min_length = 4
output_max_length = 16
output_length_sampler = LengthSampler(output_min_length, output_max_length)


generation_kwargs = {
    "min_length": -1,
    "top_k": 0.0,
    "top_p": 1.0,
    "do_sample": True,
    "pad_token_id": tokenizer.eos_token_id,
}

param_lambda = 0.5

for epoch, batch in tqdm(enumerate(ppo_trainer.dataloader)):
    query_tensors = batch["input_ids"]

    #### Get response from gpt2
    response_tensors = []
    for query in query_tensors:
        gen_len = output_length_sampler()
        generation_kwargs["max_new_tokens"] = gen_len
        response = ppo_trainer.generate(query, **generation_kwargs)
        response_tensors.append(response.squeeze()[-gen_len:])
    batch["response"] = [tokenizer.decode(r.squeeze()) for r in response_tensors]

    #### Compute sentiment score
    texts = [q + r for q, r in zip(batch["query"], batch["response"])]
    pipe_outputs1 = sentiment_pipe(texts, **sent_kwargs)
    pipe_outputs2 = sentiment_pipe1(texts, **sent_kwargs)

    scores = [[pipe_outputs1[ind][1]["score"], pipe_outputs2[ind][1]["score"]] for ind in range(len(pipe_outputs1))]

    ## mean scores
    #rewards = [torch.tensor(s).mean(dim=0)  for s in scores]

    ## Uncertainty weighted optimizatio
    rewards = [torch.tensor(s).mean(dim=0) - param_lambda*torch.tensor(s).std(dim=0) for s in scores]

    #### Run PPO step
    stats = ppo_trainer.step(query_tensors, response_tensors, rewards)
    ppo_trainer.log_stats(stats, batch, rewards)

0it [00:00, ?it/s]

In [61]:
[ torch.tensor(s).mean(dim=0) for s in scores]

[tensor(1.3511),
 tensor(2.2657),
 tensor(1.9542),
 tensor(1.2679),
 tensor(1.8520),
 tensor(1.8753),
 tensor(1.5749),
 tensor(0.3942),
 tensor(0.7615),
 tensor(-1.9742),
 tensor(1.4323),
 tensor(0.5572),
 tensor(0.8438),
 tensor(0.3842),
 tensor(0.5993),
 tensor(1.7446),
 tensor(1.7322),
 tensor(-0.6675),
 tensor(1.7866),
 tensor(1.4664),
 tensor(-2.4002),
 tensor(-0.4572),
 tensor(2.2631),
 tensor(-0.3678),
 tensor(0.3896),
 tensor(1.2680),
 tensor(-1.1418),
 tensor(2.0233),
 tensor(0.6250),
 tensor(0.7169),
 tensor(1.0640),
 tensor(2.0852),
 tensor(2.0095),
 tensor(1.0673),
 tensor(1.0639),
 tensor(2.0888),
 tensor(0.5871),
 tensor(1.2086),
 tensor(0.6123),
 tensor(1.0531),
 tensor(1.2714),
 tensor(-0.8716),
 tensor(-1.9418),
 tensor(1.8664),
 tensor(0.8826),
 tensor(1.1065),
 tensor(0.9451),
 tensor(1.6029),
 tensor(-0.7631),
 tensor(0.8570),
 tensor(0.4068),
 tensor(1.2871),
 tensor(0.5749),
 tensor(0.7306),
 tensor(1.4834),
 tensor(0.4924),
 tensor(1.0197),
 tensor(-2.1140),
 ten

In [53]:
scores = [[pipe_outputs1[ind][1]["score"], pipe_outputs2[ind][1]["score"]] for ind in range(len(pipe_outputs1))]

rewards = torch.tensor(scores).mean(dim=1)
# Assuming you have a tensor named 'tensor'
list_of_tensors = rewards.tolist()

list_of_tensors

[1.765561819076538,
 1.4367839097976685,
 -1.3036656379699707,
 0.43795913457870483,
 -1.3250980377197266,
 -2.2311089038848877,
 1.3200995922088623,
 0.7353010177612305,
 1.1265051364898682,
 -0.6398150324821472,
 1.4958276748657227,
 -1.2330360412597656,
 0.2589424252510071,
 -1.2061372995376587,
 -0.533473551273346,
 1.6424236297607422,
 1.7181966304779053,
 0.7289050817489624,
 0.4377467930316925,
 1.8524730205535889,
 -1.8193843364715576,
 0.525845468044281,
 1.3357716798782349,
 1.6599301099777222,
 0.5502035021781921,
 -0.8797567486763,
 -2.068617820739746,
 0.9794983267784119,
 -0.21824219822883606,
 -0.7660068273544312,
 -1.6335006952285767,
 0.7673906683921814,
 0.8420900106430054,
 2.062904119491577,
 2.0531365871429443,
 1.8825550079345703,
 2.239469289779663,
 -1.9163490533828735,
 -1.1391018629074097,
 -0.253135085105896,
 1.6905195713043213,
 1.4539309740066528,
 -0.4656485915184021,
 1.1018706560134888,
 1.425995111465454,
 -0.3361194431781769,
 0.950605034828186,
 -0.1

### Training progress
If you are tracking the training progress with Weights&Biases you should see a plot similar to the one below. Check out the interactive sample report on wandb.ai: [link](https://app.wandb.ai/huggingface/trl-showcase/runs/1jtvxb1m/).

<div style="text-align: center">
<img src='https://huggingface.co/datasets/trl-internal-testing/example-images/resolve/main/images/gpt2_tuning_progress.png' width='800'>
<p style="text-align: center;"> <b>Figure:</b> Reward mean and distribution evolution during training. </p>
</div>

One can observe how the model starts to generate more positive outputs after a few optimisation steps.

> Note: Investigating the KL-divergence will probably show that at this point the model has not converged to the target KL-divergence, yet. To get there would require longer training or starting with a higher initial coefficient.

## Model inspection
Let's inspect some examples from the IMDB dataset. We can use `model_ref` to compare the tuned model `model` against the model before optimisation.

In [19]:
#### get a batch from the dataset
bs = 16
game_data = dict()
dataset.set_format("pandas")
df_batch = dataset[:].sample(bs)
game_data["query"] = df_batch["query"].tolist()
query_tensors = df_batch["input_ids"].tolist()

response_tensors_ref, response_tensors = [], []

#### get response from gpt2 and gpt2_ref
for i in range(bs):
    gen_len = output_length_sampler()
    output = ref_model.generate(
        torch.tensor(query_tensors[i]).unsqueeze(dim=0).to(device), max_new_tokens=gen_len, **gen_kwargs
    ).squeeze()[-gen_len:]
    response_tensors_ref.append(output)
    output = model.generate(
        torch.tensor(query_tensors[i]).unsqueeze(dim=0).to(device), max_new_tokens=gen_len, **gen_kwargs
    ).squeeze()[-gen_len:]
    response_tensors.append(output)

#### decode responses
game_data["response (before)"] = [tokenizer.decode(response_tensors_ref[i]) for i in range(bs)]
game_data["response (after)"] = [tokenizer.decode(response_tensors[i]) for i in range(bs)]

#### sentiment analysis of query/response pairs before/after
texts = [q + r for q, r in zip(game_data["query"], game_data["response (before)"])]
game_data["rewards (before)"] = [output[1]["score"] for output in sentiment_pipe(texts, **sent_kwargs)]

texts = [q + r for q, r in zip(game_data["query"], game_data["response (after)"])]
game_data["rewards (after)"] = [output[1]["score"] for output in sentiment_pipe(texts, **sent_kwargs)]

# store results in a dataframe
df_results = pd.DataFrame(game_data)
df_results

c:\Users\arvin\anaconda3\envs\dh_lab\lib\site-packages\transformers\pipelines\text_classification.py:105: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


,query,response (before),response (after),rewards (before),rewards (after)
0,1st watched 5/27,/27/98<|endoftext|>,/82) IMDB,0.028346,0.450167
1,i must say this movie,wasn't that good at all,was always a shock to me,-2.405393,2.509559
2,This film actually manages to be mindless,goof-offs for,", boring, and",-2.352671,-2.870517
3,in a not so conventional sense,) and which even a young Ryan Beale could appr...,"that it's fifteen years late, Denzel Washingt...",1.602918,-0.982768
4,I love this movie.,Every word you utter and,It's fun to read,2.828364,2.735020
5,"Martin Weisz,","taught that, that",you are either playing,0.656042,-0.074747
6,I was,"eight years old, wonderful of anyone to have ...",definitely gonna try!!! It might be my favori...,2.625557,2.097893
7,Terrible movie.,Terrible movie.<|endoftext|>,I can't believe he wrote anything. Do yoursel...,-2.913998,-2.777426
8,The sequel to,"'The Childkillers,' starring Carole Lomb","2001: A Space Odyssey described as ""the",0.315910,1.161687
9,I guess that,out of all the previous films I've seen with ...,"this movie was made in 1932, maybe it was mad...",1.364934,-0.613506


Looking at the reward mean/median of the generated sequences we observe a significant difference.

In [20]:
print("mean:")
display(df_results[["rewards (before)", "rewards (after)"]].mean())
print()
print("median:")
display(df_results[["rewards (before)", "rewards (after)"]].median())

mean:


rewards (before)    0.345186
rewards (after)     0.401240
dtype: float64


median:


rewards (before)    0.768574
rewards (after)     0.724450
dtype: float64

## Save model
Finally, we save the model and push it to the Hugging Face for later usage.

In [21]:
model.save_pretrained("gpt2-imdb-pos-v2", push_to_hub=True)
tokenizer.save_pretrained("gpt2-imdb-pos-v2", push_to_hub=True)

ValueError: Token is required (write-access action) but no token found. You need to provide a token or be logged in to Hugging Face with `huggingface-cli login` or `huggingface_hub.login`. See https://huggingface.co/settings/tokens.